# **Driving Drowsiness Detection** - Validation

----------
*Master's in Automation and Robotics - ETSII (UPM)*\
**Subject:** Computer Vision\
**Course:** 2023-24\
**Student Name (ID):**
- Ivonne Quishpe (23146)
- Gustavo Maldonado (23102)
- Jorge Guijarro (23075)
- Micaela Cabrera (23023)
- Josep Mª Barberá (17048)

**Date:** December, 6

------------
This Notebook uses the drowsiness detection system on a set of short videos to validate its correct operation. The test set used comes from this [GitHub repository](https://github.com/EsraKavalci/Sivas-University-of-Science-and-Technology-Driver-Drowsiness-Dataset/tree/main/SUST%20Driver%20Drowsiness%20Dataset). For simplicity only 100 videos of each type have been used. The results are shown in the readme as well as in the final report of this project.

## Libraries

In [ ]:
try:
  if DATA_IS_LOADED:
    pass
except:
    !pip install mediapipe
    !wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task
DATA_IS_LOADED = True
from google.colab.patches import cv2_imshow
import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
from mediapipe import solutions
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
from google.colab import drive
import glob
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
eye_landmarks_num = 6   # 4 or 6

if eye_landmarks_num == 4:
    left_landmarks = [362, 386, 263, 374]
    right_landmarks = [33, 158, 133, 153]
else:
    left_landmarks = [362, 384, 386, 263, 374, 381]
    right_landmarks = [33, 160, 158, 133, 153, 144]
    if eye_landmarks_num != 6:
        print("Number of landmarks per eye should be 4 or 6. Setting it to 6 by default...")

mouth_landmarks = [78, 81, 13, 311, 308, 402, 14, 178]

In [ ]:
def get_face_points(face_landmarks):
    # Save the landmark coordinates for  the eyes
    right_eye_points = []
    for landmark in right_landmarks:
        right_eye_points.append([face_landmarks[landmark].x,
                            face_landmarks[landmark].y,
                            face_landmarks[landmark].z])

    left_eye_points = []
    for landmark in left_landmarks:
        left_eye_points.append([face_landmarks[landmark].x,
                            face_landmarks[landmark].y,
                            face_landmarks[landmark].z])

    # Save the landmark coordinates for  the mouth
    mouth_points = []
    for landmark in mouth_landmarks:
        mouth_points.append([face_landmarks[landmark].x,
                            face_landmarks[landmark].y,
                            face_landmarks[landmark].z])
    return right_eye_points, left_eye_points, mouth_points

In [ ]:
def get_EARs(l_p, r_p):
    if eye_landmarks_num == 4:
        p2_p4 = np.linalg.norm(l_p[1]-l_p[3])
        p1_p3 = np.linalg.norm(l_p[0]-l_p[2])
        left_EAR = (p2_p4)/(p1_p3)

        p2_p4 = np.linalg.norm(r_p[1]-r_p[3])
        p1_p3 = np.linalg.norm(r_p[0]-r_p[2])
        right_EAR = (p2_p4)/(p1_p3)
    else:
        p2_p6 = np.linalg.norm(l_p[1]-l_p[5])
        p3_p5 = np.linalg.norm(l_p[2]-l_p[4])
        p1_p4 = np.linalg.norm(l_p[0]-l_p[3])
        left_EAR = (p2_p6 + p3_p5)/(2*p1_p4)

        p2_p6 = np.linalg.norm(r_p[1]-r_p[5])
        p3_p5 = np.linalg.norm(r_p[2]-r_p[4])
        p1_p4 = np.linalg.norm(r_p[0]-r_p[3])
        right_EAR = (p2_p6 + p3_p5)/(2*p1_p4)

    return left_EAR, right_EAR

In [ ]:
def get_MAR(m_p):
  p2_p8 = np.linalg.norm(m_p[1]-m_p[7])
  p3_p7= np.linalg.norm(m_p[2]-m_p[6])
  p4_p6 = np.linalg.norm(m_p[3]-m_p[5])
  p1_p5 = np.linalg.norm(m_p[0]-m_p[4])
  MAR = (p2_p8 + p3_p7 + p4_p6)/(2*p1_p5)
  return MAR

In [ ]:
def check_drowsiness(left_EAR, right_EAR, MAR, data):
    left_score, right_score, mouth_score, eyes_tag, yawn_tag, alert, frames_sleep, frames_awake = data

    ## Increase score if eyes closed, decrease otherwise
    if (left_EAR < 0.15):
        left_score += 1
    else:
        left_score -= 1

    if (right_EAR < 0.15):
        right_score += 1
    else:
        right_score -= 1

    ## Increase score if mouth open, decrease otherwise
    if (MAR > 0.17):
        mouth_score += 1
    else:
        mouth_score -= 1

    ## Scores can't be negative
    if (left_score < 0):
        left_score = 0
    if (right_score < 0):
        right_score = 0
    if (mouth_score < 0):
        mouth_score = 0

    ## Un parpadeo normal equivale a 9 frames (300 ms si se toman 30 fps).
    if (left_score > 12 and right_score > 12):
        eyes_tag = "closed"
    else:
        eyes_tag = "open"

    ## Si la boca está abierta más de tres segundos se considera bostezo
    if (mouth_score > 70):
        yawn_tag = True
    else:
        yawn_tag = False

    if (eyes_tag == "closed" or yawn_tag == True):
        alert += 1
        frames_sleep += 1
    else:
        alert -= 1
        if (alert < 0):
            alert = 0
        frames_awake += 1
    return [left_score, right_score, mouth_score, eyes_tag, yawn_tag, alert, frames_sleep, frames_awake]

In [ ]:
def face_mesh(path):
    # path = "/home/josep/Webcam/my.webm"
    cap = cv2.VideoCapture(path)
    index_list = mouth_landmarks + left_landmarks + right_landmarks

    BaseOptions = mp.tasks.BaseOptions
    FaceLandmarker = mp.tasks.vision.FaceLandmarker
    FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
    VisionRunningMode = mp.tasks.vision.RunningMode
    options = FaceLandmarkerOptions(
        base_options=BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task'),
        running_mode=VisionRunningMode.IMAGE)

    i = 0
    prev_left_EAR, prev_right_EAR, prev_MAR = 0,0,0
    data = [0, 0, 0, 0, 0, 0, 0, 0]
    alert_time = 0
    alert_i = 0
    right_eye_points = [[0.5324623584747314, 0.5747805833816528, 0.004818260669708252],
                        [0.5445109009742737, 0.5637452006340027, -0.0011025648564100266],
                        [0.5702856183052063, 0.5580060482025146, -0.0021580627653747797],
                        [0.5986989736557007, 0.5627603530883789, 0.023030918091535568],
                        [0.5674318075180054, 0.5806047916412354, 0.004358564969152212],
                        [0.5431403517723083, 0.5771798491477966, 0.00291746249422431]]

    left_eye_points =  [[0.37691813707351685, 0.5854962468147278, 0.002876535290852189],
                        [0.3939114511013031, 0.5759596824645996, -0.014068366028368473],
                        [0.4192018210887909, 0.5710769891738892, -0.016808023676276207],
                        [0.4473620355129242, 0.5828512907028198, -0.004112580791115761],
                        [0.42484530806541443, 0.592927098274231, -0.010263157077133656],
                        [0.3993476927280426, 0.597295880317688, -0.0077733625657856464]]
    mouth_points = [[0.46936848759651184, 0.8040176630020142, 0.005104257259517908],
                    [0.49690064787864685, 0.7995496392250061, -0.012885571457445621],
                    [0.5246973037719727, 0.7988075017929077, -0.0167536623775959],
                    [0.5486900210380554, 0.7920270562171936, -0.006666502915322781],
                    [0.572444498538971, 0.7885196208953857, 0.017038118094205856],
                    [0.5485737919807434, 0.7921735644340515, -0.0028128675185143948],
                    [0.5245987176895142, 0.7992886304855347, -0.012132205069065094],
                    [0.4968000650405884, 0.8000173568725586, -0.009343835525214672]]

    with FaceLandmarker.create_from_options(options) as landmarker:
        while True:
            i += 1
            ret, frame = cap.read()
            if ret == False:
                break
            height, width, _ = frame.shape
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
            results = landmarker.detect(mp_image)

            if results.face_landmarks is not None:
                face_landmarks_list = results.face_landmarks
                for idx in range(len(face_landmarks_list)):
                    face_landmarks = face_landmarks_list[idx]
                    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
                    face_landmarks_proto.landmark.extend([
                        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks])
                    for index in index_list:
                        x = int(face_landmarks[index].x * width)
                        y = int(face_landmarks[index].y * height)
                        cv2.circle(frame, (x, y), 2, (0, 255, 0), 1)

            try:
                right_eye_points, left_eye_points, mouth_points = get_face_points(results.face_landmarks[0])
            except:
                right_eye_points = right_eye_points
                left_eye_points = left_eye_points
                mouth_points = mouth_points

            left_EAR, right_EAR = get_EARs(np.array(left_eye_points), np.array(right_eye_points))
            MAR = get_MAR(np.array(mouth_points))
            # print(left_EAR, right_EAR, MAR)

            data = check_drowsiness(left_EAR, right_EAR, MAR, data)
            _, _, _, eyes_tag, yawn_tag, alert, frames_sleep, frames_awake = data

            if (alert - alert_i > 0):
                alert_time += 1
                alert_i = alert
            # cv2_imshow(frame)
            # print(alert, alert - alert_i)

    cap.release()
    cv2.destroyAllWindows()

    return alert_time

In [ ]:
path_files = r"/content/drive/MyDrive/Github_Videos/*"
folders = glob.glob(path_files)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
results = []
for folder in folders:
  files = glob.glob(folder + "/*.mp4")
  for my_file in files:
    result = face_mesh(my_file)
    results.append(result)
    print(f"Processing file: {my_file} with results= {result}\n")